# Try to convert TF official maskrcnn to TFLite

Let's try to follow the TFLite Exercise to convert our baseline so we can try to deploy it on ElasticBeanStalk with Flask.

## 1. Pull and build the full model

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

print("Tensorflow version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
tf.config.list_physical_devices('GPU')

Tensorflow version:  2.8.0
Eager mode:  True
Hub version:  0.12.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Clone the official models repo and install object_detection

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3335, done.
remote: Counting objects: 100% (3335/3335), done.
remote: Compressing objects: 100% (2763/2763), done.
remote: Total 3335 (delta 885), reused 1379 (delta 522), pack-reused 0
Receiving objects: 100% (3335/3335), 34.32 MiB | 24.00 MiB/s, done.
Resolving deltas: 100% (885/885), done.


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1692480 sha256=f81dc5c44f96a3271123fb6960dc394ff1d5e0e2add6dcd310aeef5ef32ae2ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-j40gxxi7/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=5cae18df870166ef15ad40004a7af68b329ed86528585913a405bdc8f8b64eb4
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=07e720bff5ff017ee56ba4d148a16d633187260d9db2d9a3b1e6779babe700ad
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=09438966594cc9ed4e390369a8ff4bd7b319c7163d0ea80622

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Download and set up the model

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
!curl -O http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  462M  100  462M    0     0   162M      0  0:00:02  0:00:02 --:--:--  162M


In [ ]:
!tar zxvf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/checkpoint
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/saved_model.pb
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/assets/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables/
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables/variables.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables/variables.index


In [ ]:
!cp -rp mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint models/research/object_detection/test_data

### Get ready for inference

In [ ]:
model_name = 'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8'
pipeline_config = os.path.join('models/research/object_detection/configs/tf2/',
                                model_name + '.config')
model_dir = 'models/research/object_detection/test_data/checkpoint/'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

## 2. Run inference to make sure it works

### Download a test image

In [ ]:
!curl -O https://raw.githubusercontent.com/CSAILVision/ADE20K/main/dataset/ADE20K_2021_17_01/images/ADE/training/urban/street/ADE_train_00016869.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1079k  100 1079k    0     0  3646k      0 --:--:-- --:--:-- --:--:-- 3646k


### Prepare class labels

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

### Load model and run inference

In [ ]:
!find mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8

mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables/variables.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/variables/variables.index
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/assets
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model/saved_model.pb
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.data-00000-of-00001
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/checkpoint
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index
mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/pipeline.config


In [ ]:
model_dir = pathlib.Path('mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8')/"saved_model"
model = tf.saved_model.load(str(model_dir))

In [ ]:
segment_fn = get_model_detection_function(model)

In [ ]:
image_path = 'ADE_train_00016869.jpg'

# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
image_np = np.array(Image.open(image_path))

# Actual detection.
# input_tensor = tf.convert_to_tensor(
#     np.expand_dims(image_np, 0), dtype=tf.float32)
# detections, predictions_dict, shapes = detect_fn(input_tensor)

# output_dict = run_inference_for_single_image(model, image_np)

In [ ]:
# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image_np)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis,...]

# Run inference
model_fn = model.signatures['serving_default']
output_dict = model_fn(input_tensor)

### Examine results

In [ ]:
for k in output_dict.keys():
  print(k)

class_predictions_with_background
proposal_boxes_normalized
mask_predictions
final_anchors
raw_detection_scores
rpn_box_predictor_features
refined_box_encodings
detection_scores
rpn_objectness_predictions_with_background
raw_detection_boxes
detection_multiclass_scores
rpn_box_encodings
num_detections
detection_masks
detection_boxes
box_classifier_features
rpn_features_to_crop
detection_classes
proposal_boxes
detection_anchor_indices
anchors
image_shape
num_proposals


In [ ]:
num_detections = int(output_dict['num_detections'])
predictions = {}
predictions['num_detections'] = num_detections
for k in ['boxes', 'classes', 'masks', 'scores']:
  key = 'detection_{}'.format(k)
  predictions[key] = output_dict[key][0, :num_detections].numpy()

predictions['detection_classes'] = predictions['detection_classes'].astype(np.int64)

if 'detection_masks' in output_dict:
  # Reframe the the bbox mask to the image size.
  detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
    predictions['detection_masks'],
    predictions['detection_boxes'],
    image_np.shape[0],
    image_np.shape[1]
  )
  detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
  predictions['detection_masks_reframed'] = detection_masks_reframed.numpy()

In [ ]:
# Visualization of the results of a detection.
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    predictions['detection_boxes'],
    predictions['detection_classes'],
    predictions['detection_scores'],
    category_index,
    instance_masks=predictions.get('detection_masks_reframed', None),
    use_normalized_coordinates=True,
    line_thickness=8)

display(Image.fromarray(image_np))

Output hidden; open in https://colab.research.google.com to view.

## 3. Save the model and load it back up

I tried converting directly from step 1 above but it didn't work. Could this work, possibly? This is just like the TFLite Exercise. First we save the model, and then we load it, and then we convert to TFLite.

In [ ]:
model_save_path = './'
full_model_save_path = './saved_model'

In [ ]:
tf.saved_model.save(
  obj=model,
  export_dir=full_model_save_path
)

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


In [ ]:
%%bash -s "$full_model_save_path"
saved_model_cli show --dir "$1" --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_tensor'] tensor_info:
      dtype: DT_UINT8
      shape: (1, -1, -1, 3)
      name: serving_default_input_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['anchors'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 4)
      name: StatefulPartitionedCall:0
  outputs['box_classifier_features'] tensor_info:
      dtype: DT_FLOAT
      shape: (300, 9, 9, 1536)
      name: StatefulPartitionedCall:1
  outputs['class_predictions_with_background'] tensor_info:
      dtype: DT_FLOAT
      shape: (300, 91)
      name: StatefulPartitionedCall:2
  outputs['detection_anchor_indices'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: StatefulPartitionedCall:3
  outputs['detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 4)
      name: StatefulPartitionedCall:4
  outputs['detection_classes'] tensor_info:
      dtype: DT_FLOAT
      sha

In [ ]:
loaded = tf.saved_model.load(export_dir=full_model_save_path) # [YOUR CODE HERE]

print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'input_tensor': TensorSpec(shape=(1, None, None, 3), dtype=tf.uint8, name='input_tensor')})
{'detection_boxes': TensorSpec(shape=(1, 100, 4), dtype=tf.float32, name='detection_boxes'), 'detection_masks': TensorSpec(shape=(1, 100, 33, 33), dtype=tf.float32, name='detection_masks'), 'anchors': TensorSpec(shape=(None, 4), dtype=tf.float32, name='anchors'), 'detection_anchor_indices': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_anchor_indices'), 'rpn_box_encodings': TensorSpec(shape=(1, 49152, 4), dtype=tf.float32, name='rpn_box_encodings'), 'box_classifier_features': TensorSpec(shape=(300, 9, 9, 1536), dtype=tf.float32, name='box_classifier_features'), 'image_shape': TensorSpec(shape=(4,), dtype=tf.float32, name='image_shape'), 'detection_scores': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_scores'), 'detection_classes': TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_classes'), 'rpn_box_predictor_features': TensorSp

## 4. Convert to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir=full_model_save_path) # [YOUR CODE HERE]
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
tflite_model_save_path = "./maskrcnn.tflite"
tflite_model = converter.convert()
with open(tflite_model_save_path, "wb") as f:
    # [YOUR CODE HERE]
    f.write(tflite_model)

ConverterError: ignored

Fail!